In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import cv2
import time
import os
import sys
from pathlib import Path

# Add the challenge_solution to path
sys.path.insert(0, 'challenge_solution')

In [4]:
from challenge_solution.AIComponent import MyAIComponent

def test_ai_component():
    """Test the AI component with sample images."""
    
    print("🧪 Testing Welding Quality AI Component")
    print("=" * 50)
    
    # Initialize component
    ai_component = MyAIComponent()
    
    # Load model
    print("📦 Loading model...")
    ai_component.load_model()
    
    # Test with synthetic data if no real images available
    print("\n🔍 Testing with synthetic data...")
    
    # Create test images (224x224x3 RGB)
    test_images = [
        np.random.randint(0, 255, (224, 224, 3), dtype=np.uint8),
        np.random.randint(0, 255, (480, 640, 3), dtype=np.uint8),  # Different resolution
        np.random.randint(0, 255, (300, 400, 3), dtype=np.uint8)
    ]
    
    # Test metadata (as specified in challenge)
    test_metadata = [
        {"welding-seams": "c102", "resolution": ["224", "224"]},
        {"welding-seams": "c20", "resolution": ["640", "480"]},
        {"welding-seams": "c33", "resolution": ["400", "300"]}
    ]
    
    # Run prediction
    print(f"🚀 Running prediction on {len(test_images)} images...")
    
    start_time = time.time()
    results = ai_component.predict(test_images, test_metadata)
    total_time = time.time() - start_time
    
    # Display results
    print(f"\n📊 RESULTS:")
    print(f"Total time: {total_time:.3f}s")
    print(f"Average per image: {total_time/len(test_images)*1000:.1f}ms")
    
    for i in range(len(test_images)):
        pred = results['predictions'][i]
        probs = results['probabilities'][i]
        ood = results['OOD_scores'][i]
        
        print(f"\nImage {i+1}:")
        print(f"  Prediction: {pred}")
        print(f"  Probabilities: OK={probs[0]:.3f}, KO={probs[1]:.3f}, UNKNOWN={probs[2]:.3f}")
        print(f"  OOD Score: {ood:.3f}")
        
        # Check compliance
        time_ok = (total_time/len(test_images)) < (1/12)  # < 83.33ms
        print(f"  Time compliance: {'✅' if time_ok else '❌'}")
    
    # Validate output format
    print(f"\n🔍 VALIDATION:")
    assert isinstance(results, dict), "Result must be a dictionary"
    assert 'predictions' in results, "Must have 'predictions' key"
    assert 'probabilities' in results, "Must have 'probabilities' key"
    assert 'OOD_scores' in results, "Must have 'OOD_scores' key"
    
    assert len(results['predictions']) == len(test_images), "Wrong number of predictions"
    assert len(results['probabilities']) == len(test_images), "Wrong number of probabilities"
    assert len(results['OOD_scores']) == len(test_images), "Wrong number of OOD scores"
    
    for i, pred in enumerate(results['predictions']):
        assert pred in ['OK', 'KO', 'UNKNOWN'], f"Invalid prediction: {pred}"
        assert len(results['probabilities'][i]) == 3, f"Probabilities must have 3 elements"
        assert abs(sum(results['probabilities'][i]) - 1.0) < 0.01, f"Probabilities must sum to 1"
        assert results['OOD_scores'][i] >= 0, f"OOD score must be non-negative"
    
    print("✅ All validations passed!")
    return results

def test_with_real_images(image_folder):
    """Test with real welding images if available."""
    
    if not os.path.exists(image_folder):
        print(f"⚠️  Image folder {image_folder} not found, skipping real image test")
        return
    
    print(f"\n🖼️  Testing with real images from {image_folder}")
    
    # Load some sample images
    image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    
    if not image_files:
        print("No image files found")
        return
    
    # Take first 3 images
    sample_files = image_files[:3]
    test_images = []
    
    for file in sample_files:
        img_path = os.path.join(image_folder, file)
        img = cv2.imread(img_path)
        if img is not None:
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            test_images.append(img_rgb)
    
    if not test_images:
        print("Could not load any images")
        return
    
    # Initialize component
    ai_component = MyAIComponent()
    ai_component.load_model()
    
    # Test metadata
    test_metadata = [{"welding-seams": "c102", "resolution": ["1920", "1080"]}] * len(test_images)
    
    # Run prediction
    start_time = time.time()
    results = ai_component.predict(test_images, test_metadata)
    total_time = time.time() - start_time
    
    print(f"Results on real images:")
    print(f"Average time per image: {total_time/len(test_images)*1000:.1f}ms")
    
    for i, file in enumerate(sample_files[:len(test_images)]):
        pred = results['predictions'][i]
        probs = results['probabilities'][i]
        ood = results['OOD_scores'][i]
        
        print(f"\n{file}:")
        print(f"  Prediction: {pred}")
        print(f"  Confidence: {max(probs):.3f}")
        print(f"  OOD Score: {ood:.3f}")

test_ai_component()

# Test with real images if available
# Update this path to your actual image folder
image_folder = "../notebooks_cache/challenge-welding/datasets/welding-detection-challenge-dataset/c102/KO/expert/"
test_with_real_images(image_folder)

image_folder = "../notebooks_cache/challenge-welding/datasets/welding-detection-challenge-dataset/c102/OK/expert/"
test_with_real_images(image_folder)

print(f"\n🎉 Component testing completed successfully!")

🧪 Testing Welding Quality AI Component
📦 Loading model...
🔧 Loading Welding Quality AI Component...
Missing keys (dans le modèle mais pas dans le checkpoint) : []
Unexpected keys (dans le checkpoint mais pas dans le modèle) : []
✅ Model weights loaded from /scratch/projet_csia/challenge_trustworthy/kevin/Projet_ECE/Challenge-Welding-Reference-Solution-1/challenge_solution/best_model.pth
🔧 Using CUDA
🔥 Model warmup completed
✅ AI Component loaded on cuda

🔍 Testing with synthetic data...
🚀 Running prediction on 3 images...

📊 RESULTS:
Total time: 0.087s
Average per image: 29.0ms

Image 1:
  Prediction: KO
  Probabilities: OK=0.000, KO=1.000, UNKNOWN=0.000
  OOD Score: 0.000
  Time compliance: ✅

Image 2:
  Prediction: KO
  Probabilities: OK=0.000, KO=1.000, UNKNOWN=0.000
  OOD Score: 0.000
  Time compliance: ✅

Image 3:
  Prediction: KO
  Probabilities: OK=0.000, KO=1.000, UNKNOWN=0.000
  OOD Score: 0.000
  Time compliance: ✅

🔍 VALIDATION:
✅ All validations passed!

🖼️  Testing with re

In [8]:
from torchvision import transforms
from torchvision.transforms import InterpolationMode
sys.path.insert(0, '/home/kevin.pasini/projet_explo/kevin/uqmodels/abench/')
import challenge_solution.df_utils as dm
from challenge_solution.torch_dataloader import ImageDataFrameDataset

# Exemple de transform basique Redimensionne et normalise
transform = transforms.Compose([transforms.Resize(size=(224, 224), interpolation=InterpolationMode.BILINEAR, max_size=None, antialias=True),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

#Genere un meta dataframe utilisé pour acceder au données.
df_data = dm.explore_csv_hierarchy('../notebooks_cache',depth_name_list=['folder_1','folder_2','folder_3','seam','decision','type_label'],allowed_ext='.jpeg')
mapping = {'OK': 0, 'KO': 1}
df_data['label'] = df_data['decision'].map(mapping)
df_train,df_val = dm.stratified_train_val_split(df_data, ['seam','decision'], alpha=0.95, random_state=42)

Train_Dataset = ImageDataFrameDataset(df=df_train,root_dir="../Challenge-Welding-Reference-Solution-1/",path_col="path",label_col="label",transform=transform,channels_first=True)
Val_Dataset = ImageDataFrameDataset(df=df_train,root_dir="../Challenge-Welding-Reference-Solution-1/",path_col="path",label_col="label",transform=transform,channels_first=True)

ai_component = MyAIComponent()
ai_component.init_model()
ai_component.train_model(Train_Dataset,
                         Val_Dataset,
                         device='cpu',
                         save_path="best_model.pth",
                         augmentation_fn=None,
                         preprocess_fn=None,
                         epochs=1,
                         batch_size=64,
                         lr=3e-4)

🟦 Training started...


Training:   1%|          | 2/338 [01:29<4:09:20, 44.53s/it, loss=1.09]


KeyboardInterrupt: 